In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, Attention, DecoderAttnRNN
from train_eval import train_and_eval, inspect_model, count_parameters, summarize_results, plot_single_learning_curve
import importlib
import pickle as pkl 

In [2]:
# parameters and hyperparameters 
SRC_LANG = 'vi'
TARG_LANG = 'en'
SRC_VOCAB_SIZE = 30000
TARG_VOCAB_SIZE = 30000
BATCH_SIZE = 32
SRC_MAX_SENTENCE_LEN = 10 
TARG_MAX_SENTENCE_LEN = 10
NUM_EPOCHS = 2000 
LR = 0.0005
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 300 
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM

In [3]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

A vocabulary of 30000 is generated from a set of 35525 unique tokens.
21110 vocab tokens are not in word2vec, comprising 37.4% of entire corpus.
A vocabulary of 30000 is generated from a set of 47857 unique tokens.
8596 vocab tokens are not in word2vec, comprising 6.1% of entire corpus.


In [5]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [6]:
# create dataloaders 
full_loaders = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
fast_loaders = create_dataloaders(limited_data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [7]:
# define model 

encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                     pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [9]:
train_and_eval(model=model, full_loaders=full_loaders, fast_loaders=fast_loaders, id2token=vocab[TARG_LANG]['id2token'],
               learning_rate=LR, num_epochs=NUM_EPOCHS, print_intermediate=True, save_checkpoint=True, model_name='test2', 
               lazy_eval=False, lazy_train=True, inspect=True)

Epoch: 0.00, Train Loss: 1.58, Val Loss: 11.86, Train BLEU: 9.31, Val BLEU: 0.63
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote of of . euphore is over : they a a year molecules it it it to to to and and and weight the to : have a a year and and and and to intergovernmental almost and and the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and and and weight the the rachel when when wrote of of of is is headlines headlines they 15,000 one the the the the : t

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the intergovernmental thousand and and the the the the rachel when wrote of of is is is is headlines they 15,000 a the the the , the : they a a year and it it it pike they almost every the the the the the
Epoch: 8.00, Train Loss: 0.82, Val Loss: 12.18, Train BLEU: 9.75, Val BLEU: 1.20
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind when wrote francisco of . euphore is over : they a a of it it it it to to thousand and and the the perform perform : h

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and the the the the in rachel when wrote of of . is is is headlines they 15,000 a the the , in in : they they a year it it world world pike they almost every the the the in looking
Epoch: 16.00, Train Loss: 0.58, Val Loss: 12.50, Train BLEU: 10.09, Val BLEU: 1.18
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote of of . euphore is over : they a a it it it we we to to almost and weight to perform perform perform : have a year 

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the to thousand and the the the in in rachel when wrote of of . is is is headlines 620 15,000 a the the the in in : they a a year and and world world pike they almost and weight the the in in
Epoch: 24.00, Train Loss: 0.55, Val Loss: 12.69, Train BLEU: 9.92, Val BLEU: 1.25
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of . . euphore is over : the a year it it it we we to to almost and weight the perform perform in : have a year and 

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and weight the the the in rachel when wrote of of . is is is headlines they 15,000 a a the , in in : they a a year it it it world pike they almost every weight the smog our in
Epoch: 32.00, Train Loss: 0.48, Val Loss: 12.86, Train BLEU: 10.24, Val BLEU: 1.31
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of . . euphore is over : have a a it it it we we to to thousand and weight the perform perform in : have a year and 

...

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and weight because perform perform looking rachel when when wrote of of . . is headlines they 15,000 one &apos;s because , to . : they pages a year it it models world pike they almost every weight the smog our looking
Epoch: 270.00, Train Loss: 0.02, Val Loss: 14.47, Train BLEU: 10.36, Val BLEU: 0.65
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote francisco francisco francisco francisco francisco francisco : they a year mol

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and weight because perform perform looking rachel when when wrote of of . . is headlines they 15,000 one &apos;s because , weeks . : they pages a year it it models world pike they almost every weight the smog our looking
Epoch: 277.00, Train Loss: 0.02, Val Loss: 14.49, Train BLEU: 10.36, Val BLEU: 0.72
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote francisco francisco francisco francisco francisco francisco : they a year 

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and weight because perform perform looking rachel when when wrote of of . . is headlines they 15,000 one &apos;s because , weeks . : they pages a year it it models world pike they almost every weight the smog our looking
Epoch: 284.00, Train Loss: 0.02, Val Loss: 14.51, Train BLEU: 10.36, Val BLEU: 0.72
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote francisco francisco francisco francisco francisco francisco : they a year 

Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and weight because perform perform looking rachel when when wrote of of . . is headlines they 15,000 one &apos;s because , weeks . : they pages a year and it models world pike they almost every weight the smog our looking
Epoch: 291.00, Train Loss: 0.01, Val Loss: 14.54, Train BLEU: 10.36, Val BLEU: 0.65
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are wrote francisco francisco francisco francisco francisco francisco : they a year

KeyboardInterrupt: 